### Import knihoven a konfigurace

In [ ]:
from improutils import *

import os
import io

import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

/shared-libs/python3.8/py/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/venv/lib/python3.8/site-packages/qrdet/qrdet.py:125: UserWarning: QRDetector has been updated to use the new YoloV8 model. Use legacy=True when calling detect for backwards compatibility with 1.x versions. Or update to new output (new output is a tuple of dicts, containing several new information (1.x output is accessible through 'bbox_xyxy' and 'confidence').Forget this message if you are reading it from QReader. [This is a first download warning and will be removed at 2.1]
  warn("QRDetector has been updated to use the new YoloV8 model. Use legacy=True when calling detect "


## Úkol

Výchozím bodem k tomuto úkolu je [cvičení na perspektivní transformaci](https://courses.fit.cvut.cz/BI-SVZ/tutorials/files/6/perspective-measuring-length.html).

### První část - rovnání dokumentu a následná aplikace OCR
Práce probíhá se souborem `ocr_img.jpg` na kterém se nachází dokument známých rozměrů (papír A4). Dokument je potřeba transformovat na kolmý pohled a následně využít OCR k automatizovanému přečtení textu. Za tuto část je možné získat maximálně **3 body** a **až 1 prémiový**.

![](ocr_img.jpg)

**1) Načtěte obrázek, zobrazte ho a uložte si pole cílových korespondečních souřadnic rohů dokumentu.**

In [ ]:
image = ...

real_world_pts = ...

**2) Pomocí oblíbeného grafického editoru vyberte obrazové souřadnice rohů dokumentu ve snímku.**  
Souřadnice musí korespondovat pořadím, takže pokud je levý roh první v předchozím poli, musí být první vybrán i v tomto kroku.

In [ ]:
image_pts = ...

if (len(image_pts) != len(real_world_pts)):
    raise ValueError('Real world points and image points must be the same size.')

print('format: x, y \n', image_pts)

plot_images(image)
for i, pt in enumerate(image_pts):
    plt.annotate(str(i) + " - " + str(pt), xy=pt, textcoords="offset points", xytext=(10,10), arrowprops=dict(arrowstyle="->"))

**3) Vypočtěte transformační matici H, vypište její tvar.**

In [ ]:
H, mask = ...
print(H)

**4) Popište rozdíl mezi [`cv2.getPerspectiveTransform()`](https://docs.opencv.org/3.4.1/da/d54/group__imgproc__transform.html#ga8c1ae0e3589a9d77fffc962c49b22043) a [`cv2.findHomography`](https://docs.opencv.org/3.4.1/d9/d0c/group__calib3d.html#ga4abc2ece9fab9398f2e560d53c8c9780).**

.

**5) Využijte funkci [`cv2.warpPerspective`](https://docs.opencv.org/3.4.1/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87) k vytvoření kolmého pohledu dokumentu, výsledek zobrazte a uložte.**

In [ ]:
...

**6) Segmentujte text z "narovnaného" obrázku**
Vyberte vhodnou segmentační funkci z [modulu segmentation v improutils](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/segmentation/segmentation.py).

In [ ]:
img_bin = ...

**7) Detekujte text na "narovnaném" obraze, text vypište a uložte do textového souboru.**

Pro detekci využijte funkci [ocr](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/recognition/ocr.py#L7).

In [ ]:
text = ...(img_bin, lang="ces")

In [ ]:
...

#### Bonus k první části
Jak je vidět, výsledky po aplikaci OCR nejsou zcela uspokojivé. Je však v nutné brát potaz, že vstupní obraz není kromě geometrické transformace nijak dále předzpracován.

**8) Pokuste se vstupní obraz před aplikací OCR kvalitněji předzpracovat pomocí probíraných technik - crop, segmentace, zaostření, rozmazání, jas, kontrast, apod. Fantazii se meze nekladou. Stručně diskutujte poznatky a vlivy jednotlivých operací na výslednou kvalitu. <span style="color:blue">(0.5 bodu)</span>**


**9) Upravte krok 2) tak, aby souřadnice rohů papíru byly detekovány automaticky. <span style="color:blue">(1 bod)</span>**

**Pro zobrazení nápovědy rozklikněte tohle markdown pole.** Pokud na to chcete přijít samy, ignorujte... 
<!-- 

Hints: 
PLOT EACH STEP, it really helps!

-> gaussian blur
-> canny
-> findContours(RETR_EXTERNAL, CHAIN_APPROX_SIMPLE)
-> find largest contour by contourArea(converHull(contour)) since the FIT logo will likely be detected as a contour
-> approxPolyDP(contour, eps)
-> simplify poly approximation down to 4 points:

Find centroid of points (average of points), 
sort points by their angle: np.arctan2(*(point-centroid).

Iterate over points, always keeping track of 2 neighbouring points.

Calculate distance of middle point from the line connecting its neighbouring points.

 left = (i-1)%n 
 mid = i  
 right = (i+1) % n
 __________________________________
| left                 right      |
|  x--------------------x         |
|            ^                    |
|            |                    |
|            | distance           |
|            x                    |
|           mid                   |
|_________________________________|

Find 4 points with the largest distance. 
The idea is, that if the distance is small, the 3 points are likely on a single line.
Therefore the midpoint can be omitted.

Points with large distance are likely corners of the paper.

-> The 4 points can now be rotated, np.roll can be helpful in this case.
-> Good job, you're basically a computer vision wizard at this point!

-->